In [ ]:
from pathlib import Path
import pandas as pd

data_path = Path(input('Input the folder path:').replace('\\','/'))

for i in data_path.glob('*Segment.xlsx'):
    segment_lookup = pd.read_excel(i)

for i in data_path.glob('*.csv'):
    if i.name.__contains__("Backlog"):
        backlog_df = pd.read_csv(i ,low_memory= False)
    elif i.name.__contains__("Open"):
        orders_df = pd.read_csv(i ,low_memory= False)
    elif i.name.__contains__("Forecast"):
        forecast_df = pd.read_csv(i ,header= None ,low_memory= False)
        forecast_df.drop(index=0, axis= 'rows',inplace= True)
    else:
        pass


# Open Orders and Forcast Dataset

    ## Combining the two files ['Open Order Report - Bookings (49 columns) D11 to D21','Forecast Order History (49 columns) - D11 to D21'].
forecast_df.columns = list(orders_df.columns)
orders_forecast_df = pd.concat([orders_df,forecast_df],ignore_index=True)

    ## Adding mmm-yy in Filler column referencing 'schedule ship date'.
orders_forecast_df['Filler'] = pd.to_datetime(orders_forecast_df['schedule ship date'], format= '%m/%d/%y').dt.strftime('%b-%y').str.upper().apply(lambda x: f'="{x}"')

    ## Adding column for segment.
orders_forecast_df = orders_forecast_df.merge(right=segment_lookup, how= 'left', on= ['ect_region','application_code'])
orders_forecast_df.loc[orders_forecast_df['customer_type'] == 'Internal' ,'Segment'] = 'Intercompany'

# Shipment Dataset

    ## Replacing the column name 'World Area' to 'ect_region'.
backlog_df.rename(columns= {'World Area':'ect_region'},inplace= True)

    ## Adding column for segment.
backlog_df = backlog_df.merge(right=segment_lookup, how= 'left', on= ['ect_region','application_code'])
backlog_df.loc[backlog_df['customer_type'] == 'Internal' ,'Segment'] = 'Intercompany'

backlog_df.to_csv(data_path / '0_Clean_Shipments.csv',index= False)
orders_forecast_df.to_csv(data_path /'0_Clean_Orders and Forecast.csv',index= False)


C:\Users\E1460340\AppData\Local\Temp\ipykernel_33472\994786134.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  orders_forecast_df = pd.concat([orders_df,forecast_df],ignore_index=True)
